In [179]:
import os
import sys
import urllib.request
import datetime
import time
import json

### 뉴스 데이터 가져오기

In [180]:
client_id = 'x9KOW5id6lADbA41jdlT'
client_secret = 'A5c7fR_aYm'

In [181]:
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [182]:
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = getRequestUrl(url) #[CODE 1]

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [183]:
def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 'org_link':org_link, 'link': org_link, 'pDate':pDate})
    return

In [184]:
node = 'news' #크롤링할 대상
srcText = input('검색어를 입력하세요: ')
cnt = 0
jsonResult = []
jsonResponse = getNaverSearch(node, srcText, 1, 100) #[CODE 2]
total = jsonResponse['total']

while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
    for post in jsonResponse['items']:
        cnt += 1
        getPostData(post, jsonResult, cnt) #[CODE 3]
    
    start = jsonResponse['start'] + jsonResponse['display']
    jsonResponse = getNaverSearch(node, srcText, start, 100) #[CODE 2]

print('전체 검색 : %d 건' %total)

with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
    jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii = False)
    outfile.write(jsonFile)

print("가져온 데이터 : %d 건" %(cnt))
print('%s_naver_%s.json SAVED' % (srcText, node))

검색어를 입력하세요: 현대차
[2022-04-04 10:34:23.149412] Url Request Success
[2022-04-04 10:34:24.460856] Url Request Success
[2022-04-04 10:34:25.773396] Url Request Success
[2022-04-04 10:34:27.025245] Url Request Success
[2022-04-04 10:34:28.296600] Url Request Success
[2022-04-04 10:34:29.545858] Url Request Success
[2022-04-04 10:34:30.850688] Url Request Success
[2022-04-04 10:34:32.156380] Url Request Success
[2022-04-04 10:34:33.518127] Url Request Success
[2022-04-04 10:34:34.825564] Url Request Success
HTTP Error 400: Bad Request
[2022-04-04 10:34:35.811341] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%ED%98%84%EB%8C%80%EC%B0%A8&start=1001&display=100
전체 검색 : 1324688 건
가져온 데이터 : 1000 건
현대차_naver_news.json SAVED


### 가져온 데이터를 데이터 프레임화 하기

In [185]:
import pandas as pd
import re

In [186]:
def clean_html(x):
  #https://predictor-ver1.tistory.com/4
  x = re.sub("\&\w*\;","",x)
  x = re.sub("<.*?>","",x)
  return x

In [187]:
df_json = pd.DataFrame(jsonResult)
df_json

,cnt,title,description,org_link,link,pDate
0,1,"포항시, 수도권 로봇기업 유치 이어 차세대 대한민국 로봇산업 육성","또, 국민의 삶의 질을 향상시키는 핵심분야로서 반도체 이후 POST 핵심산업으로 귀...",https://www.gukjenews.com/news/articleView.htm...,https://www.gukjenews.com/news/articleView.htm...,2022-04-04 19:08:00
1,2,포스코 오늘 전원 출근…기업들 “재택근무→일상 복귀?”,"<b>현대차</b>, SK, LG 등은 기존 지침에 변화는 없다고 밝혔습니다. 탄력...",https://news.kbs.co.kr/news/view.do?ncd=543153...,https://news.kbs.co.kr/news/view.do?ncd=543153...,2022-04-04 19:04:00
2,3,"도로교통공단-현대엔지니어링, 중대재해 교통사고 예방 업무협약",도로교통공단은 현대엔지니어링이 관리하는 <b>현대차</b> 그룹 통근버스 운영 46...,https://www.news1.kr/articles/?4637492,https://www.news1.kr/articles/?4637492,2022-04-04 18:50:00
3,4,"BMW -7%, 제네시스 -4%…잘나가던 중고차, 고유가에 ‘휘청’",중고차 매매 사이트 엔카닷컴에선 지난달 <b>현대차</b> 투싼(가솔린 1.6터보)...,https://www.joongang.co.kr/article/25060819,https://www.joongang.co.kr/article/25060819,2022-04-04 18:34:00
4,5,"[아유경제_재건축] 잠실우성1ㆍ2ㆍ3차 재건축, 정비계획 변경 '순항'",또한 영동대로 개발 및 <b>현대차</b> 신사옥(GBC) 건축 등 주변의 개발 호...,http://www.areyou.co.kr/news/articleView.html?...,http://www.areyou.co.kr/news/articleView.html?...,2022-04-04 18:34:00
...,...,...,...,...,...,...
995,996,"<b>현대차</b> 전주공장 직원, 기계에 끼어 사망…고용부 조사 착수",고용노동부 등에 따르면 이날 전북 완주군 <b>현대차</b> 전주공장에서 작업 중이...,http://news.mt.co.kr/mtview.php?no=20220331150...,http://news.mt.co.kr/mtview.php?no=20220331150...,2022-03-31 15:39:00
996,997,"<b>현대차</b> '아이오닉5', 美 카즈닷컴 '최고의 가족용 전기차' 선정",31일 <b>현대차</b>에 따르면 아이오닉5는 '카즈닷컴'이 29일(현지시간) 발...,http://www.dizzotv.com/site/data/html_dir/2022...,http://www.dizzotv.com/site/data/html_dir/2022...,2022-03-31 15:38:00
997,998,현대자동차 전주공장서 사망사고 발생...중대재해 조사중,고용노동부는 31일 오후 1시 10분경 전북 완주군에 위치한 <b>현대차</b> 전...,https://idsn.co.kr/news/view/1065593451521653,https://idsn.co.kr/news/view/1065593451521653,2022-03-31 15:38:00
998,999,브라질 전략형 소형 해치백 <b>현대차</b> ‘HB20’,<b>현대차</b> 브라질 전용 소형 해치백 HB20의 페이스리프트 프로토타입이 포...,http://www.thedrive.co.kr/news/newsview.php?nc...,http://www.thedrive.co.kr/news/newsview.php?nc...,2022-03-31 15:36:00


In [188]:
df_json['title'] = df_json['title'].apply(lambda x: clean_html(x))
df_json['description'] = df_json['description'].apply(lambda x: clean_html(x))
df_json

,cnt,title,description,org_link,link,pDate
0,1,"포항시, 수도권 로봇기업 유치 이어 차세대 대한민국 로봇산업 육성","또, 국민의 삶의 질을 향상시키는 핵심분야로서 반도체 이후 POST 핵심산업으로 귀...",https://www.gukjenews.com/news/articleView.htm...,https://www.gukjenews.com/news/articleView.htm...,2022-04-04 19:08:00
1,2,포스코 오늘 전원 출근…기업들 “재택근무→일상 복귀?”,"현대차, SK, LG 등은 기존 지침에 변화는 없다고 밝혔습니다. 탄력적 운영은 가...",https://news.kbs.co.kr/news/view.do?ncd=543153...,https://news.kbs.co.kr/news/view.do?ncd=543153...,2022-04-04 19:04:00
2,3,"도로교통공단-현대엔지니어링, 중대재해 교통사고 예방 업무협약",도로교통공단은 현대엔지니어링이 관리하는 현대차 그룹 통근버스 운영 46개사 약 11...,https://www.news1.kr/articles/?4637492,https://www.news1.kr/articles/?4637492,2022-04-04 18:50:00
3,4,"BMW -7%, 제네시스 -4%…잘나가던 중고차, 고유가에 ‘휘청’",중고차 매매 사이트 엔카닷컴에선 지난달 현대차 투싼(가솔린 1.6터보) 중고차가 3...,https://www.joongang.co.kr/article/25060819,https://www.joongang.co.kr/article/25060819,2022-04-04 18:34:00
4,5,"[아유경제_재건축] 잠실우성1ㆍ2ㆍ3차 재건축, 정비계획 변경 '순항'",또한 영동대로 개발 및 현대차 신사옥(GBC) 건축 등 주변의 개발 호재가 이어져 ...,http://www.areyou.co.kr/news/articleView.html?...,http://www.areyou.co.kr/news/articleView.html?...,2022-04-04 18:34:00
...,...,...,...,...,...,...
995,996,"현대차 전주공장 직원, 기계에 끼어 사망…고용부 조사 착수",고용노동부 등에 따르면 이날 전북 완주군 현대차 전주공장에서 작업 중이던 A씨(41...,http://news.mt.co.kr/mtview.php?no=20220331150...,http://news.mt.co.kr/mtview.php?no=20220331150...,2022-03-31 15:39:00
996,997,"현대차 '아이오닉5', 美 카즈닷컴 '최고의 가족용 전기차' 선정",31일 현대차에 따르면 아이오닉5는 '카즈닷컴'이 29일(현지시간) 발표한 '202...,http://www.dizzotv.com/site/data/html_dir/2022...,http://www.dizzotv.com/site/data/html_dir/2022...,2022-03-31 15:38:00
997,998,현대자동차 전주공장서 사망사고 발생...중대재해 조사중,고용노동부는 31일 오후 1시 10분경 전북 완주군에 위치한 현대차 전주공장서 사망...,https://idsn.co.kr/news/view/1065593451521653,https://idsn.co.kr/news/view/1065593451521653,2022-03-31 15:38:00
998,999,브라질 전략형 소형 해치백 현대차 ‘HB20’,현대차 브라질 전용 소형 해치백 HB20의 페이스리프트 프로토타입이 포착됐다. 외신...,http://www.thedrive.co.kr/news/newsview.php?nc...,http://www.thedrive.co.kr/news/newsview.php?nc...,2022-03-31 15:36:00


In [189]:
df_json['description']

0      또, 국민의 삶의 질을 향상시키는 핵심분야로서 반도체 이후 POST 핵심산업으로 귀...
1      현대차, SK, LG 등은 기존 지침에 변화는 없다고 밝혔습니다. 탄력적 운영은 가...
2      도로교통공단은 현대엔지니어링이 관리하는 현대차 그룹 통근버스 운영 46개사 약 11...
3      중고차 매매 사이트 엔카닷컴에선 지난달 현대차 투싼(가솔린 1.6터보) 중고차가 3...
4      또한 영동대로 개발 및 현대차 신사옥(GBC) 건축 등 주변의 개발 호재가 이어져 ...
                             ...                        
995    고용노동부 등에 따르면 이날 전북 완주군 현대차 전주공장에서 작업 중이던 A씨(41...
996    31일 현대차에 따르면 아이오닉5는 '카즈닷컴'이 29일(현지시간) 발표한 '202...
997    고용노동부는 31일 오후 1시 10분경 전북 완주군에 위치한 현대차 전주공장서 사망...
998    현대차 브라질 전용 소형 해치백 HB20의 페이스리프트 프로토타입이 포착됐다. 외신...
999    현대차·기아에서 제작·판매한 ‘아반떼’ 등 8개 차종 2878대는 앞좌석 안전띠 조...
Name: description, Length: 1000, dtype: object

In [190]:
description_list = df_json['description'].values.tolist()
description_slice = []
for i in range(0, len(description_list)):
    description_slice += description_list[i].split(' ')
len(description_slice)

25245

In [191]:
title_list = df_json['title'].values.tolist()
title_slice = []
for i in range(0, len(title_list)):
    title_slice += title_list[i].split(' ')
len(title_slice)

7298

In [192]:
desCount = {} 
 
for word in description_slice:
    desCount[word] = desCount.get(word, 0) + 1
    desKeys = sorted(desCount.keys())

In [193]:
titleCount = {} 
 
for title in title_slice:
    titleCount[title] = titleCount.get(title, 0) + 1
    titleKeys = sorted(titleCount.keys())

In [194]:
for word in desKeys:
    if desCount[word] > 50:
        print(word + ':' + str(desCount[word]))

:955
1분기:71
1일:94
31일:78
3월:102
가장:73
감소한:60
것으로:66
관계자는:53
국내:106
글로벌:71
대비:93
대한:64
등:216
등이:57
따르면:124
및:69
반도체:61
생산:68
시가총액:58
시장에서:94
오후:53
올해:70
유럽:67
있다.:114
자동차:128
작업:60
전기차:75
전년:68
전주공장에서:71
정몽구:65
중:72
지난:71
지난달:89
지난해:57
총:54
현대차:522
현대차가:53
현대차는:183
현대차와:66
현대차의:64


In [195]:
for word in titleKeys:
    if titleCount[word] > 20:
        print(word + ':' + str(titleCount[word]))

1분기:43
3월:103
40대:35
감소:24
국내:23
글로벌:24
노동자:60
마감:30
비교:21
사망:31
상승:24
선정:23
역대:24
완성차:22
위반:30
육성:21
전기차:32
전주공장:33
전주공장서:49
정몽구:22
조사:40
중:39
차세대:21
첫:22
코스피:32
코스피,:75
판매:84
판매량:23
폭스바겐:33
하락:29
현대차:211
현대차,:86
현대차·기아,:23
